# 第二语言学习

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tnrange, tqdm_notebook
import random
df = pd.read_csv("result_data2.csv")

In [ ]:
GDP_increase_rate = [0.02,0.07,0.03]
limitation_of_Num = 1e6  #不考虑的语言限制
group_num = 1e5  #计算群体的个数


LOM = limitation_of_Num
Countrys = df["Country"]
Languages = ["Chinese","English","Hindi","Urdu","Spanish","Arabic","Malay","Russian","Bengali","Portuguese",\
             "French","Hausa","Punjabi","Japanese","German","Persian"]
Internet = {"English":1,"Russian":0.13,"German":0.11,"Japanese":0.10,"Spanish"\
                     :0.10,"French":0.08,"Portuguese":0.05,"Italian":0.05,"Chinese":0.04,"Hindi":0,\
                      "Urdu":0,"Arabic":0.01,"Malay":0,"Bengali":0,"Persian":0.03,"Hausa":0,"Punjabi":0}
Academic = {"Chinese":0.24,"English":1,"Hindi":0,"Urdu":0,"Spanish":0.48,"Arabic":0.08,"Malay":0,"Russian":0.16,"Bengali":0,\
                      "Portuguese":0.09,"French":0.16,"Hausa":0,"Japanese":0.045,"German":0.06,"Persian":0,"Punjabi":0}
Top10_economics = ["Chinese","English","Hindi","French","Japanese","German"]
Diplomacy = {"Chinese":0.093,"English":1,"Hindi":0.031,"Urdu":0,"Spanish":0.468,"Arabic":0.187,"Malay":0.031,"Russian":0.187,"Bengali":0,\
                      "Portuguese":0.312,"French":0.843,"Hausa":0,"Japanese":0,"German":0.093,"Persian":0,"Punjabi":0}
developed_country = list(df[df["PPP"]>np.mean(df["PPP"])]["Country"])
backward_country = list(df[df["PPP"]<np.median(df["PPP"])]["Country"])

In [ ]:
#求出说改语言的国家数NOC和总人数NOP,以及上年GDP
def count(df):
    df2 = df.copy()
    NOP=[];NOC=[];GDP_list = []
    for language in Languages:
        #print(language)
        NOC.append(np.sum(df2[language]>LOM))
        NOP.append(np.sum(df2[df2[language]>LOM][language]))
        GDP_list.append(np.sum(df2[df2[language]>LOM]["GDP"]))
    NOC = NOC/np.max(NOC);NOP = NOP/np.max(NOP);
    MAX = np.max(GDP_list)
    for i in range(len(Languages)):
        GDP_list[i] = GDP_list[i]/ MAX
    return NOP,NOC,GDP_list

In [ ]:
#求出该国家目前已经有的language_list
def find_offical_language(df,CountryName):
    df2 = df.copy()
    official_language_list = []
    df3 = df2[df2["Country"]== CountryName]
    for language in Languages:
        if (int(df3[language])>LOM):
            official_language_list.append(language)
    return official_language_list

In [ ]:
def Find_language(rank,num,temp_df):
    for i in range(len(rank)):
        if (rank[i] > num):
            break
    language = temp_df.index[i]
    return language,i

In [ ]:
#求出对该国家几种语言的评价情况
def ComprehensiveEvaluation(official_language_list,NOC,NOP,GDP_list):
    i=0;w=[];w1=[];w2=[];w3=[];w4=[];w5=[];
    for language in Languages:
        if (language in official_language_list):
            w1_a=1
        else:
            w1_a=0
        w1_b = Internet[language]
        w1.append(w1_a*0.5+w1_b*0.5)
        w2.append(NOC[i]*0.5+NOP[i]*0.5)
        if (language in Top10_economics):
            w3_a = 1
        else:
            w3_a = 0
        w3_b = GDP_list[i]
        w3.append(w3_a *0.5 + w3_b *0.5)
        w4.append(Academic[language])
        w5.append(Diplomacy[language])
        i=i+1
    for i in range(len(Languages)):
        w.append(w1[i]*0.3416+w2[i]*0.1967+w3[i]*0.3416+w4[i]*0.0732+w5[i]*0.0469)
    w = w/np.sum(w)
    return w

In [ ]:
def language_to_learn(df,CountryName):
    GDP_this_year = int(df[df["Country"] == CountryName].GDP/(df[df["Country"] == CountryName].Population))
    if (GDP_this_year>average_GDP):
        possbility1 = 0.4
    elif (GDP_this_year>=median_GDP)&(GDP_this_year<=average_GDP):
        possbility1 = 0.6
    elif (GDP_this_year<median_GDP):
        possbility1 = 0.2
    else:
        possbility1 = 0 
    a = df[df["Country"] == CountryName][Languages]>limitation_of_Num
    a = a.T
    diversity = int((np.sum(a)))
    possbility2 = diversity/16
    possbility = 0.5*possbility1+0.5*possbility2
    return possbility

In [ ]:
def findNative(df,CountryName):
    data3 = df[df["Country"]==CountryName]
    data3 = data3[Languages]
    #a= list(data3)
    a= np.array(data3)
    a =a.tolist()
    a=a[0]
    index_lang = a.index(max(a))
    lang = Languages[index_lang]
    return lang

In [ ]:
(NOP,NOC,GDP_list) = count(df)#求出说某语言的国家数NOC和总人数NOP,以及上年GDP
kk=0
df_new = df.copy()#以下均在df_new上操作
GDP_average_list = [] #求人均GDP
for country in Countrys:
    ppp = int((df_new[df_new["Country"] == country].GDP)/(df_new[df_new["Country"] == country].Population)*1e4) #单位万美元
    GDP_average_list.append(ppp)
average_GDP = np.mean(GDP_average_list)
median_GDP = np.median(GDP_average_list)
for CountryName in tqdm_notebook(Countrys):
    #print(CountryName)
    official_language_list = find_offical_language(df,CountryName);
    w = ComprehensiveEvaluation(official_language_list,NOC,NOP,GDP_list)
    temp_df = pd.DataFrame(np.arange(len(Languages)).reshape((len(Languages),1)),index= Languages ,columns=['W'])
    for i in range(len(Languages)):
        temp_df["W"].iloc[i] = w[i]
    temp_df = temp_df.sort_values("W",ascending = False)
    for Num in range(10):
        temp_df.iloc[-Num-1] = 0
    rank = temp_df["W"]
    rank2 = [rank[0]]
    for i in range(len(rank)-1):
        rank2.append(rank2[-1]+rank[i+1])
    temp_df["W"]=rank2 # here we got the distribution
    #print(temp_df)
    population_this_year =  int(df[df["Country"] == CountryName]["Population"])
    rate =  float(df[df["Country"] == CountryName]["Net I-rate"])
    possbility = language_to_learn(df,CountryName)
    Nest_year_language = [0]*len(Languages)
    POP = int(population_this_year * rate * 0.01 * possbility)
    
    GnerNo = int(np.floor(POP/group_num)+1)
    for i in range(GnerNo):
        max_percent = int(100*np.max(rank2))
        randnum = random.randint(0,max_percent)/100
        (languageToLearn,index) = Find_language(rank2,randnum,temp_df)
        if (POP>group_num):
            Nest_year_language[index] += group_num
            POP = POP - group_num
        else:
            Nest_year_language[index] += int(POP)
    newborn_dic = {}
    native_lang = findNative(df,CountryName)
    for i in range(len(Languages)):
        ab = temp_df.index[i]
        newborn_dic[temp_df.index[i]] = Nest_year_language[i] + int(df[df["Country"]== CountryName][ab])
    if CountryName in developed_country:
        increase_rate = GDP_increase_rate[0]
    elif CountryName in backward_country:
        increase_rate = GDP_increase_rate[2]
    else:
        increase_rate = GDP_increase_rate[1]
    GDP_new = int((df[df["Country"] == CountryName]["GDP"]) * (increase_rate+1))
    newborn_dic[native_lang] += POP
    population_next_year = int(population_this_year * (rate*0.01+1)) 
    df_new.drop(df_new[df_new["Country"]==CountryName].index,inplace = True)
    df_new.loc[130+kk] = {"Country": CountryName,"GDP":GDP_new,"Population":population_next_year,"PPP":0,"Net I-rate":rate,\
                                    "Total":0,"Chinese":newborn_dic["Chinese"],"English":newborn_dic["English"],"Hindi":newborn_dic["Hindi"],\
                                     "Urdu":newborn_dic["Urdu"],"Spanish":newborn_dic["Spanish"],"Arabic":newborn_dic["Arabic"],"Malay":newborn_dic["Malay"],\
                                     "Russian":newborn_dic["Russian"],"Bengali":newborn_dic["Bengali"], "Portuguese":newborn_dic["Portuguese"],"French":newborn_dic["French"],\
                                     "Hausa":newborn_dic["Hausa"],"Punjabi":newborn_dic["Punjabi"],"Japanese":newborn_dic["Japanese"],"German":newborn_dic["German"],"Persian":newborn_dic["Persian"]}
    kk=kk+1
df_new.reset_index(drop=True)
df_new.to_csv("1.csv",index = False)

In [ ]:
for year in range(50):
    Name1 = str(year+1)+".csv"
    df = pd.read_csv(Name1)
    (NOP,NOC,GDP_list) = count(df)
    kk=0
    df_new = df.copy()
    GDP_average_list = [] #求人均GDP
    for country in Countrys:
        ppp = int((df_new[df_new["Country"] == country].GDP)/(df_new[df_new["Country"] == country].Population)*1e4) #单位万美元
        GDP_average_list.append(ppp)
    #print(GDP_average_list)
    average_GDP = np.mean(GDP_average_list);#print(average_GDP)
    median_GDP = np.median(GDP_average_list);#print(median_GDP)
    for CountryName in tqdm_notebook(Countrys):
        official_language_list = find_offical_language(df,CountryName);
        w = ComprehensiveEvaluation(official_language_list,NOC,NOP,GDP_list)
        temp_df = pd.DataFrame(np.arange(len(Languages)).reshape((len(Languages),1)),index= Languages ,columns=['W'])
        for i in range(len(Languages)):
            temp_df["W"].iloc[i] = w[i]
        temp_df = temp_df.sort_values("W",ascending = False)
        for Num in range(10):
            temp_df.iloc[-Num-1] = 0
        rank = temp_df["W"]
        rank2 = [rank[0]]
        for i in range(len(rank)-1):
            rank2.append(rank2[-1]+rank[i+1])
        temp_df["W"]=rank2 # here we got the distribution
        population_this_year =  int(df[df["Country"] == CountryName]["Population"])
        rate =  float(df[df["Country"] == CountryName]["Net I-rate"])

        Nest_year_language = [0]*len(Languages)
        POP = int(population_this_year * rate * 0.01)
        GnerNo = int(np.floor(POP/group_num)+1)
        for i in range(GnerNo):
            max_percent = int(100*np.max(rank2))
            randnum = random.randint(0,max_percent)/100
            (languageToLearn,index) = Find_language(rank2,randnum,temp_df)
            if (POP>group_num):
                Nest_year_language[index] += group_num
                POP = POP - group_num
            else:
                Nest_year_language[index] += int(POP)
        newborn_dic = {}
        native_lang = findNative(df,CountryName)
        for i in range(len(Languages)):
            ab = temp_df.index[i]
            newborn_dic[temp_df.index[i]] = Nest_year_language[i] + int(df[df["Country"]== CountryName][ab])
        if CountryName in developed_country:
            increase_rate = GDP_increase_rate[0]
        elif CountryName in backward_country:
            increase_rate = GDP_increase_rate[2]
        else:
            increase_rate = GDP_increase_rate[1]
        GDP_new = int((df[df["Country"] == CountryName]["GDP"]) * (increase_rate+1))
        newborn_dic[native_lang] += POP
        population_next_year = int(population_this_year * (rate*0.01+1))
        df_new.drop(df_new[df_new["Country"]==CountryName].index,inplace = True)
        df_new.loc[130+kk] = {"Country": CountryName,"GDP":GDP_new,"Population":population_next_year,"PPP":0,"Net I-rate":rate,\
                                        "Total":0,"Chinese":newborn_dic["Chinese"],"English":newborn_dic["English"],"Hindi":newborn_dic["Hindi"],\
                                         "Urdu":newborn_dic["Urdu"],"Spanish":newborn_dic["Spanish"],"Arabic":newborn_dic["Arabic"],"Malay":newborn_dic["Malay"],\
                                         "Russian":newborn_dic["Russian"],"Bengali":newborn_dic["Bengali"], "Portuguese":newborn_dic["Portuguese"],"French":newborn_dic["French"],\
                                         "Hausa":newborn_dic["Hausa"],"Punjabi":newborn_dic["Punjabi"],"Japanese":newborn_dic["Japanese"],"German":newborn_dic["German"],"Persian":newborn_dic["Persian"]}
        kk=kk+1
    df_new.reset_index(drop=True)
    Name2 = str(year+2)+".csv"
    df_new.to_csv(Name2,index = False)